In [3]:
!pip install docling -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import os
from groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

In [21]:
load_dotenv('.env')  
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [2]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_size = len(embedding_model.embed_query("test"))

qdrant = QdrantClient(":memory:")
qdrant.recreate_collection(
    collection_name="documents",
    vectors_config={"size": vector_size, "distance": "Cosine"}
)
print("✅ Embedding model and Qdrant initialized")


/tmp/ipykernel_53352/4187742279.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Embedding model and Qdrant initialized


/tmp/ipykernel_53352/4187742279.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


In [5]:
from docling.document_converter import DocumentConverter


# document per local path or URL
source = "../pdf/จังหวัดน่าน.pdf"  

converter = DocumentConverter()
result = converter.convert(source)


In [6]:
print(result.document.export_to_markdown())

## จังหวัดน่าน

น่าน เป็นจังหวัดหนึ่งในประเทศไทย ตั้งอยู่ทางทิศตะวันออกสุดของภาคเหนือ เป็นที่ตั้ง ของเมืองที่สำคัญในอดีต เช่น เวียงวรนคร (เมืองพลัว) เวียงศีรษะเกษ (เมืองงั่ว) เวียงภูเพียงแช่ แห้ง อีกทั้งยังเป็นแหล่งต้นน้ำของแม่น้ำน่าน

ประวัติศาสตร์  มีประวัติความเป็นมาที่เก่าแก่ยาวนาน มีชื่อเรียกในพงศาวดารว่า นันท บุรี เมืองน่านในอดีตเป็นนครรัฐเล็ก ๆ ก่อตัวขึ้นราวกลางพุทธศตวรรษที่ 18 บริเวณที่ราบลุ่ม แม่น้ำน่านและแม่น้ำสาขาในหุบเขาทางตะวันออกของภาคเหนือ

## สมัยเมืองล่าง-วรนคร

ประวัติศาสตร์เมืองน่านเริ่มปรากฏขึ้นราว พ.ศ. 1825 ภายใต้การนำของพญาภูคาและ นางพญาจำปาผู้เป็นชายา ซึ่งทั้งสองเป็นชาวเมืองเงินยาง ได้เป็นแกนนำพาผู้คนอพยพมาตั้ง ศูนย์การปกครองอยู่ที่เมืองล่าง ต่อมาเพี้ยนเป็นเมืองย่าง (เชื่อกันว่าคือบริเวณริมฝั่งด้านใต้ของ แม่น้ำย่างบริเวณตำบลศิลาเพชร อำเภอปัว เลยไปถึงลำน้ำบั่ว ใกล้ทิวเขาดอยภูคาในเขตบ้าน เสี้ยว บ้านทุ่งฆ้อง บ้านลอมกลาง ตำบลยม อำเภอท่าวังผา) เพราะปรากฏร่องรอยชุมชนใน สภาพที่เป็นคูน้ำ คันดิน และกำแพงเมืองซ้อนกันอยู่ เห็นชัดเจนที่สุดคือบริเวณข้างพระธาตุ จอมพริกบ้านเสี้

In [ ]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# สมมุติว่าได้ markdown string มา
markdown_text = result.document.export_to_markdown() 
doc = Document(page_content=markdown_text)

# แบ่งข้อความ
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document_chunks = splitter.split_documents([doc])  

Created a chunk of size 932, which is longer than the specified 500
Created a chunk of size 852, which is longer than the specified 500
Created a chunk of size 655, which is longer than the specified 500
Created a chunk of size 758, which is longer than the specified 500
Created a chunk of size 1096, which is longer than the specified 500
Created a chunk of size 1122, which is longer than the specified 500


In [13]:
# Extract text from document chunks
texts = [doc.page_content for doc in document_chunks]

# Convert texts into vectors
vectors = embedding_model.embed_documents(texts)

# Upsert data into Qdrant
points = [PointStruct(id=i, vector=vectors[i], payload={"text": texts[i]}) for i in range(len(texts))]
qdrant.upsert(collection_name="documents", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
def search_documents(query):
    # Convert query to vector
    query_vector = embedding_model.embed_query(query)
    
    # Search Qdrant for similar documents
    search_results = qdrant.search(
        collection_name="documents",
        query_vector=query_vector,
        limit=4  # Retrieve top 5 relevant documents
    )
    
    # Check if results are found
    if not search_results:
        return []  # Return empty if no documents found
    
    # Extract text from results 
    return [hit.payload.get("text", "เอกสารไม่มีข้อความ") for hit in search_results]

In [ ]:
def generate_answer(query):
    # Retrieve relevant documents from Qdrant
    retrieved_docs = search_documents(query)

    # Check if any documents were retrieved
    if not retrieved_docs:
        return "ไม่พบข้อมูลที่เกี่ยวข้อง"  # "No relevant information found"

    # Create the context for the language model
    context = "\n".join([str(doc) for doc in retrieved_docs if isinstance(doc, str)])

    # Check if context has content
    if not context.strip():
        # "No relevant information found"
        return "ไม่พบข้อมูลที่เกี่ยวข้อง"

    prompt = f"ข้อมูลอ้างอิง:\n{context}\n\nคำถาม: {query}\n\nคำตอบ:"

    # Initialize Groq API client
    groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

    try:
        # Requesting completion from Groq API
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant", messages=[{"role": "user", 
                                                     "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"เกิดข้อผิดพลาดในการสร้างคำตอบ: {str(e)}"
    # "Error occurred in generating the answer"

In [24]:
# 🔥 Test the question!
query = "มีรายการที่ต้องเทียวอะไรบ้าง"  
answer = generate_answer(query)
print(answer)

/tmp/ipykernel_53352/3693313488.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


จากข้อมูลที่ให้มา มีข้อเท็จจริงที่สำคัญต่อไปนี้:

1. เจ้าเก้าเถื่อนราชบุตรขึ้นครองเมืองปัวแทนหลังขุนฟองไป
2. พญาภูคาครองเมืองย่างยาวนานและมีอายุมากขึ้น มีความประสงค์ให้เจ้าเก้าเถื่อนมาครองเมืองย่าง
3. พระเจ้าติโลกราชกษัตริย์แห่งนครเชียงใหม่ ปี พ.ศ. 1993 จัดกองทัพเข้ายึดเมืองน่านและแหล่งเกลือบ่อมาง.

เป็นเท็จจริงที่สำคัญที่สุดของข้อมูลนี้


In [25]:
# 🔥 Test the question!
query = "สรุปใจความของเอกสาร"  
answer = generate_answer(query)
print(answer)

/tmp/ipykernel_53352/3693313488.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


เอกสารนี้เล่าถึงประวัติของจังหวัดน่านโดยแบ่งออกเป็นสามยุคสมัย: 

1. สมัยรัตนโกสินทร์ : ภูมิประเทศและตำนานของจังหวัดน่าน รวมถึงการเป็นแหล่งกบดานของสมาชิกพรรคคอมมิวนิสต์แห่งประเทศไทย
2. สมัยล้านนา : การครอบครองเมืองน่านด้วยกำลังสมทบของพระเจ้าติโลกราช และการอพยพของพญาอินต๊ะแก่นท้าวออกจากเมืองน่าน

โดยรวมแล้ว เอกสารนี้เล่าถึงประวัติและภูมิประเทศของจังหวัดน่าน และการเปลี่ยนแปลงของอำนาจและความมั่งคั่งในประวัติศาสตร์จังหวัดน่าน
